In [1]:
import yaml

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from functions import (standardize_columns, drop_duplicates)

config = None  # <-- Initialize config
try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("Yaml configuration file not found!")
config
if config is not None:
    job_nyc1 = pd.read_csv(config['input_data']['file1'])
    job_nyc2 = pd.read_csv(config['input_data']['file2'])
else:
    print("Cannot load data files because config is missing.")


In [2]:
job_nyc1


,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Classification,Title Code No,Level,Job Category,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,534519,DEPT OF ENVIRONMENT PROTECTION,Internal,2,Region Supervisor,ADMINISTRATIVE PROJECT MANAGER,Competitive-1,8300B,00,"Engineering, Architecture, & Planning",...,Appointments are subject to OMB approval. For ...,To Apply: Click Apply Now button,NaN,96-05 Horace Harding Expway,NaN,There is no residency requirement for this title.,06/08/2022,NaN,06/12/2022,05/06/2024
1,631363,DEPT OF ENVIRONMENT PROTECTION,Internal,1,2024-BWS-010-Early Warning Remote Modeling Int...,SUMMER COLLEGE INTERN,Non-Competitive-5,10234,00,"Engineering, Architecture, & Planning",...,NaN,Apply click the âApply Nowâ button. DEP...,35 Hours per week,"*The position is located in Valhalla, NY, in W...",NaN,There is no New York City residency requiremen...,04/02/2024,11-JUN-2024,04/02/2024,05/06/2024
2,628158,BRONX DISTRICT ATTORNEY,External,2,Intelligence Analyst,COMMUNITY ASSOCIATE,Non-Competitive-5,56057,00,Communications & Intergovernmental Affairs Tec...,...,NaN,"For City employees, to complete your applicati...",NaN,NaN,NaN,City Residency is not required for this position,02/27/2024,26-FEB-2025,02/27/2024,05/06/2024
3,629002,OFFICE OF CRIMINAL JUSTICE,Internal,1,"Senior Coordinator, Capital Projects",COMMUNITY COORDINATOR,Non-Competitive-5,56058,00,"Engineering, Architecture, & Planning",...,NaN,NaN,NaN,NaN,NaN,New York City residency is generally required ...,03/05/2024,29-MAY-2024,04/29/2024,05/06/2024
4,613253,DEPT OF DESIGN & CONSTRUCTION,Internal,1,Director,ADMINISTRATIVE CITY PLANNER,Competitive-1,10053,M2,"Engineering, Architecture, & Planning",...,NaN,Please go to NYC Government Jobs Explore Care...,NaN,NaN,NaN,New York City residency is generally required ...,11/06/2023,NaN,03/05/2024,05/06/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5064,609088,DEPT OF ENVIRONMENT PROTECTION,Internal,1,Assistant Commissioner,ADMINISTRATIVE PUBLIC HEALTH S,Competitive-1,82989,M4,Administration & Human Resources Health Public...,...,NaN,Press the apply button,Monday - Friday 35 + hours per week,59-17 Junction Blvd Corona Ny,NaN,New York City residency is generally required ...,10/06/2023,NaN,10/06/2023,05/06/2024
5065,632969,DEPARTMENT OF TRANSPORTATION,Internal,3,Assistant General Counsel,AGENCY ATTORNEY,Non-Competitive-5,30087,01,Legal Affairs,...,NaN,All resumes are to be submitted electronically...,35hrs,"55 Water Street, NY, NY",NaN,New York City Residency is not required for th...,04/11/2024,NaN,04/11/2024,05/06/2024
5066,508258,DEPARTMENT OF CORRECTION,Internal,1,Agency Attorney,AGENCY ATTORNEY,Non-Competitive-5,30087,04,Legal Affairs,...,NaN,For City employees: Go to Employee Self-Servic...,NaN,NaN,NaN,New York City residency is generally required ...,12/08/2021,NaN,12/08/2021,05/06/2024
5067,543713,NYC HOUSING AUTHORITY,Internal,3,Senior Project Manager,ADMINISTRATIVE CONSTRUCTION PR,Competitive-1,82991,M4,"Administration & Human Resources Engineering, ...",...,1.\tCandidates with permanent civil service st...,Click the Apply Now button.,NaN,NaN,NaN,NYCHA has no residency requirements.,08/17/2022,NaN,11/07/2022,05/06/2024


In [3]:
job_nyc1["Preferred Skills"].isnull().sum()


np.int64(1375)

In [4]:
job_nyc2["Preferred Skills"].isnull().sum()

np.int64(2699)

In [5]:
# job_nyc["Preferred Skills"].value_counts(dropna=False)

In [6]:
print(pd.DataFrame({
    "non_nulls": job_nyc1.count(),
    "nulls": job_nyc1.isnull().sum(),
    "total": len(job_nyc1)
}))



                               non_nulls  nulls  total
Job ID                              5069      0   5069
Agency                              5069      0   5069
Posting Type                        5069      0   5069
# Of Positions                      5069      0   5069
Business Title                      5069      0   5069
Civil Service Title                 5069      0   5069
Title Classification                5069      0   5069
Title Code No                       5069      0   5069
Level                               5069      0   5069
Job Category                        5069      0   5069
Full-Time/Part-Time indicator       4990     79   5069
Career Level                        5069      0   5069
Salary Range From                   5069      0   5069
Salary Range To                     5069      0   5069
Salary Frequency                    5069      0   5069
Work Location                       5069      0   5069
Division/Work Unit                  5069      0   5069
Job Descri

In [7]:
print(pd.DataFrame({
    "non_nulls": job_nyc2.count(),
    "nulls": job_nyc2.isnull().sum(),
    "total": len(job_nyc2)
}))

                               non_nulls  nulls  total
Job ID                              6030      0   6030
Agency                              6030      0   6030
Posting Type                        6030      0   6030
# Of Positions                      6030      0   6030
Business Title                      6030      0   6030
Civil Service Title                 6030      0   6030
Title Classification                6030      0   6030
Title Code No                       6030      0   6030
Level                               6030      0   6030
Job Category                        6030      0   6030
Full-Time/Part-Time indicator       6030      0   6030
Career Level                        6030      0   6030
Salary Range From                   6030      0   6030
Salary Range To                     6030      0   6030
Salary Frequency                    6030      0   6030
Work Location                       6030      0   6030
Division/Work Unit                  6030      0   6030
Job Descri

In [8]:
merged = pd.concat([job_nyc1,job_nyc2], ignore_index = True)

In [9]:
merged = merged.drop_duplicates(subset=['Job ID'])

In [10]:
merged

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Classification,Title Code No,Level,Job Category,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,534519,DEPT OF ENVIRONMENT PROTECTION,Internal,2,Region Supervisor,ADMINISTRATIVE PROJECT MANAGER,Competitive-1,8300B,00,"Engineering, Architecture, & Planning",...,Appointments are subject to OMB approval. For ...,To Apply: Click Apply Now button,NaN,96-05 Horace Harding Expway,NaN,There is no residency requirement for this title.,06/08/2022,NaN,06/12/2022,05/06/2024
1,631363,DEPT OF ENVIRONMENT PROTECTION,Internal,1,2024-BWS-010-Early Warning Remote Modeling Int...,SUMMER COLLEGE INTERN,Non-Competitive-5,10234,00,"Engineering, Architecture, & Planning",...,NaN,Apply click the âApply Nowâ button. DEP...,35 Hours per week,"*The position is located in Valhalla, NY, in W...",NaN,There is no New York City residency requiremen...,04/02/2024,11-JUN-2024,04/02/2024,05/06/2024
2,628158,BRONX DISTRICT ATTORNEY,External,2,Intelligence Analyst,COMMUNITY ASSOCIATE,Non-Competitive-5,56057,00,Communications & Intergovernmental Affairs Tec...,...,NaN,"For City employees, to complete your applicati...",NaN,NaN,NaN,City Residency is not required for this position,02/27/2024,26-FEB-2025,02/27/2024,05/06/2024
3,629002,OFFICE OF CRIMINAL JUSTICE,Internal,1,"Senior Coordinator, Capital Projects",COMMUNITY COORDINATOR,Non-Competitive-5,56058,00,"Engineering, Architecture, & Planning",...,NaN,NaN,NaN,NaN,NaN,New York City residency is generally required ...,03/05/2024,29-MAY-2024,04/29/2024,05/06/2024
4,613253,DEPT OF DESIGN & CONSTRUCTION,Internal,1,Director,ADMINISTRATIVE CITY PLANNER,Competitive-1,10053,M2,"Engineering, Architecture, & Planning",...,NaN,Please go to NYC Government Jobs Explore Care...,NaN,NaN,NaN,New York City residency is generally required ...,11/06/2023,NaN,03/05/2024,05/06/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10906,685126,HRA/DEPT OF SOCIAL SERVICES,External,1,DEPUTY DIRECTOR OF ONBOARDING,ADM MANAGER-NON-MGRL FRM M1/M2,Competitive-1,1002C,01,Administration & Human Resources,...,The Human Resources Administration/Department ...,Click Apply Now Button,9:00 AM to 5:00 PM,NaN,NaN,New York City residency is generally required ...,10/18/2024,NaN,10/21/2024,09/16/2025
10957,711599,DEPT OF ENVIRONMENT PROTECTION,Internal,1,ENVIRONMENTAL SPECIALIST,CITY RESEARCH SCIENTIST,Non-Competitive-5,21744,03,"Engineering, Architecture, & Planning Public S...",...,NaN,NaN,NaN,NaN,NaN,New York City residency is generally required ...,04/25/2025,NaN,04/25/2025,09/16/2025
10961,715866,OFFICE OF THE MAYOR,External,1,Deputy Communications Director,Special Assistant (MA)-MGRL,Pending Classification-2,0668A,MY,Communications & Intergovernmental Affairs,...,NaN,NaN,NaN,NaN,NaN,New York City residency is generally required ...,05/29/2025,NaN,05/29/2025,09/16/2025
11037,714239,OFFICE OF THE MAYOR,External,1,Director for Digital Communications,Special Assistant (MA)-MGRL,Pending Classification-2,0668A,MY,Communications & Intergovernmental Affairs,...,NaN,NaN,NaN,NaN,NaN,City Residency is required for this position,05/12/2025,NaN,05/12/2025,09/16/2025


In [11]:
# merged.to_csv("jobdata.csv", index=False)

In [12]:
merged.to_csv(config['output_data']['file1'], index=False, sep=";", encoding="utf-8")

In [13]:
print(merged.columns)

Index(['Job ID', 'Agency', 'Posting Type', '# Of Positions', 'Business Title',
       'Civil Service Title', 'Title Classification', 'Title Code No', 'Level',
       'Job Category', 'Full-Time/Part-Time indicator', 'Career Level',
       'Salary Range From', 'Salary Range To', 'Salary Frequency',
       'Work Location', 'Division/Work Unit', 'Job Description',
       'Minimum Qual Requirements', 'Preferred Skills',
       'Additional Information', 'To Apply', 'Hours/Shift', 'Work Location 1',
       'Recruitment Contact', 'Residency Requirement', 'Posting Date',
       'Post Until', 'Posting Updated', 'Process Date'],
      dtype='object')


In [14]:
from functions import (standardize_columns, drop_duplicates)

In [15]:
merged = drop_duplicates(merged,'Job ID')

In [16]:
# Standardize column names
merged = standardize_columns(merged) 

In [17]:
merged

,Job_ID,Agency,Posting_Type,#_Of_Positions,Business_Title,Civil_Service_Title,Title_Classification,Title_Code_No,Level,Job_Category,...,Additional_Information,To_Apply,Hours/Shift,Work_Location_1,Recruitment_Contact,Residency_Requirement,Posting_Date,Post_Until,Posting_Updated,Process_Date
0,534519,DEPT OF ENVIRONMENT PROTECTION,Internal,2,Region Supervisor,ADMINISTRATIVE PROJECT MANAGER,Competitive-1,8300B,00,"Engineering, Architecture, & Planning",...,Appointments are subject to OMB approval. For ...,To Apply: Click Apply Now button,NaN,96-05 Horace Harding Expway,NaN,There is no residency requirement for this title.,06/08/2022,NaN,06/12/2022,05/06/2024
1,631363,DEPT OF ENVIRONMENT PROTECTION,Internal,1,2024-BWS-010-Early Warning Remote Modeling Int...,SUMMER COLLEGE INTERN,Non-Competitive-5,10234,00,"Engineering, Architecture, & Planning",...,NaN,Apply click the âApply Nowâ button. DEP...,35 Hours per week,"*The position is located in Valhalla, NY, in W...",NaN,There is no New York City residency requiremen...,04/02/2024,11-JUN-2024,04/02/2024,05/06/2024
2,628158,BRONX DISTRICT ATTORNEY,External,2,Intelligence Analyst,COMMUNITY ASSOCIATE,Non-Competitive-5,56057,00,Communications & Intergovernmental Affairs Tec...,...,NaN,"For City employees, to complete your applicati...",NaN,NaN,NaN,City Residency is not required for this position,02/27/2024,26-FEB-2025,02/27/2024,05/06/2024
3,629002,OFFICE OF CRIMINAL JUSTICE,Internal,1,"Senior Coordinator, Capital Projects",COMMUNITY COORDINATOR,Non-Competitive-5,56058,00,"Engineering, Architecture, & Planning",...,NaN,NaN,NaN,NaN,NaN,New York City residency is generally required ...,03/05/2024,29-MAY-2024,04/29/2024,05/06/2024
4,613253,DEPT OF DESIGN & CONSTRUCTION,Internal,1,Director,ADMINISTRATIVE CITY PLANNER,Competitive-1,10053,M2,"Engineering, Architecture, & Planning",...,NaN,Please go to NYC Government Jobs Explore Care...,NaN,NaN,NaN,New York City residency is generally required ...,11/06/2023,NaN,03/05/2024,05/06/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10906,685126,HRA/DEPT OF SOCIAL SERVICES,External,1,DEPUTY DIRECTOR OF ONBOARDING,ADM MANAGER-NON-MGRL FRM M1/M2,Competitive-1,1002C,01,Administration & Human Resources,...,The Human Resources Administration/Department ...,Click Apply Now Button,9:00 AM to 5:00 PM,NaN,NaN,New York City residency is generally required ...,10/18/2024,NaN,10/21/2024,09/16/2025
10957,711599,DEPT OF ENVIRONMENT PROTECTION,Internal,1,ENVIRONMENTAL SPECIALIST,CITY RESEARCH SCIENTIST,Non-Competitive-5,21744,03,"Engineering, Architecture, & Planning Public S...",...,NaN,NaN,NaN,NaN,NaN,New York City residency is generally required ...,04/25/2025,NaN,04/25/2025,09/16/2025
10961,715866,OFFICE OF THE MAYOR,External,1,Deputy Communications Director,Special Assistant (MA)-MGRL,Pending Classification-2,0668A,MY,Communications & Intergovernmental Affairs,...,NaN,NaN,NaN,NaN,NaN,New York City residency is generally required ...,05/29/2025,NaN,05/29/2025,09/16/2025
11037,714239,OFFICE OF THE MAYOR,External,1,Director for Digital Communications,Special Assistant (MA)-MGRL,Pending Classification-2,0668A,MY,Communications & Intergovernmental Affairs,...,NaN,NaN,NaN,NaN,NaN,City Residency is required for this position,05/12/2025,NaN,05/12/2025,09/16/2025


In [18]:
merged.to_csv("jobdata_dropped.csv", index=False)